# Quickstart: Querying PDF With Astra and LangChain

### A question-answering demo using Astra DB and LangChain, powered by Vector Search

#### Pre-requisites:

You need a **_Serverless Cassandra with Vector Search_** database on [Astra DB](https://astra.datastax.com) to run this demo. As outlined in more detail [here](https://docs.datastax.com/en/astra-serverless/docs/vector-search/quickstart.html#_prepare_for_using_your_vector_database), you should get a DB Token with role _Database Administrator_ and copy your Database ID: these connection parameters are needed momentarily.

You also need an [OpenAI API Key](https://cassio.org/start_here/#llm-access) for this demo to work.

#### What you will do:

- Setup: import dependencies, provide secrets, create the LangChain vector store;
- Run a Question-Answering loop retrieving the relevant headlines and having an LLM construct the answer.

Install the required dependencies:

In [1]:
!pip install -q cassio datasets langchain openai tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 kB 471.7 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.6/974.6 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 46.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.9/18.9 MB 45.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.8/315.8 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

Import the packages you'll need:

In [2]:
!pip install -U langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 1.8 MB/s eta 0:00:00


In [3]:
# LangChain components to use
from langchain.vectorstores.cassandra import Cassandra
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
from langchain.llms import OpenAI
from langchain.embeddings import OpenAIEmbeddings

# Support for dataset retrieval with Hugging Face
from datasets import load_dataset

# With CassIO, the engine powering the Astra DB integration in LangChain,
# you will also initialize the DB connection:
import cassio

In [4]:
!pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 2.1 MB/s eta 0:00:00


In [5]:
from PyPDF2 import PdfReader

### Setup

In [17]:
ASTRA_DB_APPLICATION_TOKEN = "" # enter the "AstraCS:..." string found in in your Token JSON file
ASTRA_DB_ID = "" # enter your Database ID

OPENAI_API_KEY = "" # enter your OpenAI key

#### Provide your secrets:

Replace the following with your Astra DB connection details and your OpenAI API key:

In [7]:
# provide the path of  pdf file/files.
pdfreader = PdfReader('budget_speech.pdf')

In [8]:
from typing_extensions import Concatenate
# read text from pdf
raw_text = ''
for i, page in enumerate(pdfreader.pages):
    content = page.extract_text()
    if content:
        raw_text += content

In [9]:
raw_text

'1 \n     \nBUDGET SPEECH 2014 BUDGET SPEECH 2014 BUDGET SPEECH 2014 BUDGET SPEECH 2014- - --15 15 15 15     \n    \nEsteemed Members of the Council, \n \nIt is a matter of great personal pride for me to prese nt before you the annual \nbudget of this prestigious Council in its Centenary yea r.  The completion of one \nhundred years is a major milestone in the history of any organization.  The \ncommemoration of the Centenary of NDMC is obviously a  time for us to \ncelebrate, but even more so, a time to celebrate our history.  The success story \nof NDMC is now a century old and marks the contributio n of so many known and \nunknown personalities who added significantly to its g rowth.  I stand here first & \nforemost to pay our tribute to the vision and dedication  of our past leaders, \nexecutives and thousands of employees, workers who ha ve worked with \nceaseless passion to build up this organization over th e past century. \n \n1.2  The last 100 years were, for NDMC, the era of 

Initialize the connection to your database:

_(do not worry if you see a few warnings, it's just that the drivers are chatty about negotiating protocol versions with the DB.)_

In [18]:
cassio.init(token=ASTRA_DB_APPLICATION_TOKEN, database_id=ASTRA_DB_ID)

ERROR:cassandra.connection:Closing connection <LibevConnection(134549299081184) ccb8b89c-bcd9-4d37-b522-efb8ee5ac989-us-east1.db.astra.datastax.com:29042:3b8afe0a-9b27-48ef-af55-d95b0aef8e45> due to protocol error: Error from server: code=000a [Protocol error] message="Beta version of the protocol used (5/v5-beta), but USE_BETA flag is unset"


Create the LangChain embedding and LLM objects for later usage:

In [19]:
llm = OpenAI(openai_api_key=OPENAI_API_KEY)
embedding = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

Create your LangChain vector store ... backed by Astra DB!

In [20]:
astra_vector_store = Cassandra(
    embedding=embedding,
    table_name="qa_mini_demo",
    session=None,
    keyspace=None,
)

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [21]:
from langchain.text_splitter import CharacterTextSplitter
# We need to split the text using Character Text Split such that it sshould not increse token size
text_splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size = 800,
    chunk_overlap  = 200,
    length_function = len,
)
texts = text_splitter.split_text(raw_text)

In [22]:
texts[:50]

['1 \n     \nBUDGET SPEECH 2014 BUDGET SPEECH 2014 BUDGET SPEECH 2014 BUDGET SPEECH 2014- - --15 15 15 15     \n    \nEsteemed Members of the Council, \n \nIt is a matter of great personal pride for me to prese nt before you the annual \nbudget of this prestigious Council in its Centenary yea r.  The completion of one \nhundred years is a major milestone in the history of any organization.  The \ncommemoration of the Centenary of NDMC is obviously a  time for us to \ncelebrate, but even more so, a time to celebrate our history.  The success story \nof NDMC is now a century old and marks the contributio n of so many known and \nunknown personalities who added significantly to its g rowth.  I stand here first & \nforemost to pay our tribute to the vision and dedication  of our past leaders,',
 "unknown personalities who added significantly to its g rowth.  I stand here first & \nforemost to pay our tribute to the vision and dedication  of our past leaders, \nexecutives and thousands of e

### Load the dataset into the vector store



In [23]:

astra_vector_store.add_texts(texts[:50])

print("Inserted %i headlines." % len(texts[:50]))

astra_vector_index = VectorStoreIndexWrapper(vectorstore=astra_vector_store)

NameError: name 'astra_vector_store' is not defined

### Run the QA cycle

Simply run the cells and ask a question -- or `quit` to stop. (you can also stop execution with the "▪" button on the top toolbar)

Here are some suggested questions:
- _What is the current GDP?_
- _How much the agriculture target will be increased to and what the focus will be_


In [ ]:
first_question = True
while True:
    if first_question:
        query_text = input("\nEnter your question (or type 'quit' to exit): ").strip()
    else:
        query_text = input("\nWhat's your next question (or type 'quit' to exit): ").strip()

    if query_text.lower() == "quit":
        break

    if query_text == "":
        continue

    first_question = False

    print("\nQUESTION: \"%s\"" % query_text)
    answer = astra_vector_index.query(query_text, llm=llm).strip()
    print("ANSWER: \"%s\"\n" % answer)

    print("FIRST DOCUMENTS BY RELEVANCE:")
    for doc, score in astra_vector_store.similarity_search_with_score(query_text, k=4):
        print("    [%0.4f] \"%s ...\"" % (score, doc.page_content[:84]))


Enter your question (or type 'quit' to exit): How much the agriculture target will be increased to and what the focus will be

QUESTION: "How much the agriculture target will be increased to and what the focus will be"


ANSWER: "The agriculture credit target will be increased to ` 20 lakh crore with focus on animal husbandry, dairy and fisheries."

FIRST DOCUMENTS BY RELEVANCE:


    [0.9327] "of Millet Research, Hyderabad  will be supported as the Centre of Excellence 
for sh ..."
    [0.9327] "of Millet Research, Hyderabad  will be supported as the Centre of Excellence 
for sh ..."
    [0.9207] "Agriculture Accelerator Fund  
17. An Agriculture Accelerator Fund will be set-up to ..."
    [0.9207] "Agriculture Accelerator Fund  
17. An Agriculture Accelerator Fund will be set-up to ..."

What's your next question (or type 'quit' to exit): What is the current GDP

QUESTION: "What is the current GDP"


ANSWER: "The current GDP is estimated to be 7 per cent."

FIRST DOCUMENTS BY RELEVANCE:


    [0.8920] "estimated to be at 7 per cent. It is notable that this is the highest among all 
the ..."
    [0.8920] "estimated to be at 7 per cent. It is notable that this is the highest among all 
the ..."
    [0.8915] "multiplier impact on growth and employment. After the subdued period of 
the pandemi ..."
    [0.8915] "multiplier impact on growth and employment. After the subdued period of 
the pandemi ..."

What's your next question (or type 'quit' to exit): quit
